In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from utils import find_elem,save_json

In [ ]:
BASE_URL = "https://osdr.nasa.gov/bio/repo/data/studies/OSD-"

wd = webdriver.Chrome()
wd.implicitly_wait(1)
time.sleep(1)
wd.maximize_window()


In [12]:
for i in range(521,877):
    link = BASE_URL+str(i)
    try:
        wd.get(link)
        element = WebDriverWait(wd, 1).until(
        EC.visibility_of_element_located((By.ID, "header-tagline"))
        )
        html = wd.page_source #extract webpage html
        soup = BeautifulSoup(html, "html.parser")
        name_block = soup.find("div",{'class':"flex-container justify-start"}) # extract block where name of OSD study is
        # extract name itself 
        study_name = name_block.select("body > app-root > repo-view > mat-sidenav-container > mat-sidenav-content > div.flex-container.justify-start > div.three-quarters-width > h1 > span:nth-child(4)")[0].text
        time.sleep(0.3)
        # extract genelabID block
        b_element = find_elem('/html/body/app-root/repo-view/mat-sidenav-container/mat-sidenav-content/div[2]/b[1]', wd) 
        # extract genelabID and replace first :
        genelab_id = wd.execute_script("return arguments[0].nextSibling.nodeValue.trim();", b_element).replace(":","").strip()
        time.sleep(0.3)

        # extract unique doi
        doi = find_elem("/html/body/app-root/repo-view/mat-sidenav-container/mat-sidenav-content/div[2]/a[1]",wd).text
        time.sleep(0.3)

        # main description of study
        description = find_elem('//*[@id="cdk-accordion-child-0"]/div/div/div/description-panel/div[1]/div/p',wd).text
        print(description)
        time.sleep(0.3)

        # which organisms this study is related to
        organisms_block = find_elem('//*[@id="cdk-accordion-child-0"]/div/div/div/description-panel/div[3]/mat-grid-list', wd)
        organisms = [organism.text for organism in organisms_block.find_elements(By.CLASS_NAME,"mat-grid-tile-content")]
        time.sleep(0.3)

        # An ontology factor is a variable or condition in an experiment that might influence the results. It’s essentially an experimental parameter.
        # These ontology links help out RAG system understand that different experiments are related even if the text uses slightly different words.
        ontology_block = find_elem('//*[@id="cdk-accordion-child-0"]/div/div/div/description-panel/div[2]/table/tbody', wd)
        ontology_block_rows = ontology_block.find_elements(By.TAG_NAME, 'tr')
        ontology_concepts = {}
        for row in ontology_block_rows:
            key_values = row.find_elements(By.TAG_NAME,"td")
            ontology_concepts[key_values[0].text] = key_values[1].text
        print(ontology_concepts)
        time.sleep(0.3)

        # extract authors related to this study(contacts)
        authors_block = find_elem('//*[@id="cdk-accordion-child-0"]/div/div/div/description-panel/div[6]/div',wd)
        authors_elems_list = authors_block.find_elements(By.CLASS_NAME,"contact-container.ng-star-inserted")

        authors = []
        for elem in authors_elems_list:
            authors.append(elem.text.replace(",","").strip())
        time.sleep(0.3)

        # We should also extract protocols which are descriptive of how experiments lasted.
        # first we expandtheir descriptions
        expand_protocols_btn = find_elem('//*[@id="cdk-accordion-child-4"]/div/div/div/protocols-panel/div/div/button[1]/span[1]', wd)
        time.sleep(1)
        expand_protocols_btn.click()
        time.sleep(1)
        # find block with all cards
        samples_block = find_elem('//*[@id="Samples"]/div/div',wd)
        samples_cards = samples_block.find_elements(By.XPATH, "./div")

        # extract name and descriptions, store in format List[dict(name:value, description:value)]
        samples_info_list = []
        for card in samples_cards:
            protocol_sample_name = card.find_element(By.CLASS_NAME,"card-header").text
            nested_div = card.find_element(By.TAG_NAME,"div").find_element(By.TAG_NAME,'div').find_element(By.TAG_NAME,'div')
            sample_description = nested_div.find_elements(By.TAG_NAME,'div')[1].text.replace("Description", "").strip()
            sample_info = {"name":protocol_sample_name, "description": sample_description}
            samples_info_list.append(sample_info)
            
        osd_study = {"doi":doi,
                    'study_name':study_name,
                    "genelab_id":genelab_id,
                    "description":description,
                    "organisms":organisms,
                    "ontology_factors":ontology_concepts,
                    "authors":authors,
                    "protocole_samples":samples_info_list,
                    "link":link}
        save_json(osd_study,"./data/osdr_raw/OSD_"+str(i))
    except Exception as e:
        print(link)
        print(e)
        

https://osdr.nasa.gov/bio/repo/data/studies/OSD-521
Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff70ca41eb5+80197]
	GetHandleVerifier [0x0x7ff70ca41f10+80288]
	(No symbol) [0x0x7ff70c7c02fa]
	(No symbol) [0x0x7ff70c817cd7]
	(No symbol) [0x0x7ff70c817f9c]
	(No symbol) [0x0x7ff70c86ba87]
	(No symbol) [0x0x7ff70c8403bf]
	(No symbol) [0x0x7ff70c8687fb]
	(No symbol) [0x0x7ff70c840153]
	(No symbol) [0x0x7ff70c808b02]
	(No symbol) [0x0x7ff70c8098d3]
	GetHandleVerifier [0x0x7ff70ccfe83d+2949837]
	GetHandleVerifier [0x0x7ff70ccf8c6a+2926330]
	GetHandleVerifier [0x0x7ff70cd186c7+3055959]
	GetHandleVerifier [0x0x7ff70ca5cfee+191102]
	GetHandleVerifier [0x0x7ff70ca650af+224063]
	GetHandleVerifier [0x0x7ff70ca4af64+117236]
	GetHandleVerifier [0x0x7ff70ca4b119+117673]
	GetHandleVerifier [0x0x7ff70ca310a8+11064]
	BaseThreadInitThunk [0x0x7ffc8a14e8d7+23]
	RtlUserThreadStart [0x0x7ffc8c328d9c+44]

Spaceflight presents a unique environment with complex stressors, including microgravity and radiation

We examined the effects of ~30 days of spaceflight on glycogen synthase kinase 3 (GSK3) content and inhibitory serine phosphorylation in murine muscle and bone samples from four separate missions (BION-M1, Rodent Research RR1, RR9, and RR18). Spaceflight reduced GSK3b content across all missions, whereas its serine phosphorylation was elevated with RR18 and BION-M1. The reduction in GSK3b was linked to the reduction in type IIA fibers commonly observed with spaceflight as these fibers are particularly enriched with GSK3. We then tested the effects of inhibiting GSK3 before this fiber type shift, and we demonstrate that muscle-specific Gsk3 knockdown increased muscle mass, preserved muscle strength, and promoted the oxidative fiber type with Earth-based hindlimb unloading. In bone, GSK3 activation was enhanced after spaceflight; and strikingly, muscle-specific Gsk3 deletion increased bone mineral density in response to hindlimb unloading. Thus, future studies should test the effects of 